In [2]:
import cv2
import json
from math import *
import mediapipe as mp
import numpy as np

In [3]:
# 어깨 관절 각도 계산
def upperArmX(sho, arm):
    return atan(abs(arm.x - sho.x) / abs(arm.y - sho.y))

# cv2 이미지 프로세스 처리 - 포즈
def simpleCV2Process_pose(image):
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# cv2 이미지 프로세스 처리 - 손
def simpleCV2Process_hands(image):
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [10]:
# 벡터간의 사잇각도 측정 - 팔 각도, 손가락 마디
# https://beanexpert.tistory.com/entry/Vision-%EB%91%90-%EC%A7%81%EC%84%A0-%EC%82%AC%EC%9D%B4%EC%9D%98-%EA%B0%81%EB%8F%84-%EC%82%AC%EC%9D%B4%EA%B0%81-%EA%B5%AC%ED%95%98%EA%B8%B0
#
# def toVec(head, tail):
#     head = [head.x, head.y]
#     tail = [tail.x, tail.y]
#     head = np.array(head); tail = np.array(tail)
#     return acos(np.dot(head, tail)/(np.linalg.norm(head)*np.linalg.norm(tail)))

def toVec(head, tail):
    return [ tail.x - head.x, tail.y - head.y, tail.z - head.z ]

def vec2Rotate(head, tail):
    vecIn = head[0]*tail[0] + head[1]*tail[1] + head[2]*tail[2]
    if vecIn == 0:
        return 0
    vec01 = sqrt(pow(head[0], 2) + pow(head[1], 2) + pow(head[2], 2))
    vec02 = sqrt(pow(tail[0], 2) + pow(tail[1], 2) + pow(tail[2], 2))
    return acos(vecIn/(vec01*vec02))

def fingerRotation(hand):
    # 왼손 오른손 구분 코드도 넣어줘야됨
    # 왼손 0 오른손 1

    fingers = [
        [[0, 1], [1, 2], [2, 3], [3, 4]],        # 엄지 손가락
        [[0, 5], [5, 6], [6, 7], [7, 8]],        # 중지 손가락
        [[0, 9], [9, 10], [10, 11], [11, 12]],   # 검지 손가락
        [[0, 13], [13, 14], [14, 15], [15, 16]], # 약지 손가락
        [[0, 17], [17, 18], [18, 19], [19, 20]]  # 소지 손가락
    ]
    fingerLines = []
    joints = []

    for finger in fingers:
        temp = []
        for fl in finger:
            temp.append(toVec(hand.landmark[fl[0]], hand.landmark[fl[1]]))
        fingerLines.append(temp)
    for fl in fingerLines:
        for joint in range(3):
            joints.append(vec2Rotate(fl[joint], fl[joint + 1]))
    return joints

In [11]:
# 사용법
# degrees(toVec(pose[10], pose[12]))

In [31]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


video = "MOV000241430_700X466"

# cap = cv2.VideoCapture("MOV000249438_700X466.webm")
cap = cv2.VideoCapture(video)
# cap = cv2.VideoCapture(0)

frames = {}
fps = 0


# pose 감지
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success: break
        image, results = simpleCV2Process_pose(image)
        # mp_drawing.draw_landmarks( image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # # 좌우 반전되어 있었음
        # # cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        # cv2.imshow('MediaPipe Pose', image)
        # if cv2.waitKey(10) & 0xFF == ord('q'):
        #     break
        frame = {}
        frame["left"] = {"upperArm" : upperArmX(results.pose_landmarks.landmark[11], results.pose_landmarks.landmark[13])}
        frame["right"] = {"upperArm" : upperArmX(results.pose_landmarks.landmark[12], results.pose_landmarks.landmark[14])}
        frames[f"frame_{fps}"] = frame
        fps += 1

cap = cv2.VideoCapture(video)
fps = 0
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        fps += 1
        success, image = cap.read()
        if not success: break
        image, results = simpleCV2Process_hands(image)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                # mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                #                           mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                #                           mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                #                           )
                # print(str(fingerRotation(hand)))
                hands_data = fingerRotation(hand)
                frame = {}
                # frame["left"] = {"hand" : {}, upperArmX(results.pose_landmarks.landmark[11], results.pose_landmarks.landmark[13])}
                frame = {
                    "Thumb" : {
                        "1" : hands_data[0],
                        "2" : hands_data[1],
                        "3" : hands_data[2],
                    },
                    "Index" : {
                        "1" : hands_data[3],
                        "2" : hands_data[4],
                        "3" : hands_data[5],
                    },
                    "Middle" : {
                        "1" : hands_data[6],
                        "2" : hands_data[7],
                        "3" : hands_data[8],
                    },
                    "Ring" : {
                        "1" : hands_data[9],
                        "2" : hands_data[10],
                        "3" : hands_data[11],
                    },
                    "Pinky" : {
                        "1" : hands_data[12],
                        "2" : hands_data[13],
                        "3" : hands_data[14],
                    }
                }
                frames[f"frame_{fps}"]["right"]["hand"] = frame
                # print(frames[f"frame_{fps}"]["right"])

with open('test.json', 'w', encoding='utf-8') as make_file:
    json.dump(frames, make_file, indent="\t")

# cap.release()
# cv2.destroyAllWindows()

In [4]:
# mp_drawing = mp.solutions.drawing_utils
# mp_drawing_styles = mp.solutions.drawing_styles
# mp_pose = mp.solutions.pose
#
# cap = cv2.VideoCapture("MOV000249438_700X466.webm")
# # cap = cv2.VideoCapture("yee.webm")
# # cap = cv2.VideoCapture(0)
#
# frames = {}
# fps = 0
#
# with mp_pose.Pose(
#         min_detection_confidence=0.5,
#         min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         success, image = cap.read()
#         if not success: break
#         image.flags.writeable = False
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         results = pose.process(image)
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#         mp_drawing.draw_landmarks( image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
#         # 좌우 반전되어있었음
#         # cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
#         cv2.imshow('MediaPipe Pose', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#
#         frame = {}
#         frame["left"] = {"upperArm" : upperArmX(results.pose_landmarks.landmark[11], results.pose_landmarks.landmark[13])}
#         frame["right"] = {"upperArm" : upperArmX(results.pose_landmarks.landmark[12], results.pose_landmarks.landmark[14])}
#         frames[f"frame_{fps}"] = frame
#         fps += 1
#
# with open('test.json', 'w', encoding='utf-8') as make_file:
#     json.dump(frames, make_file, indent="\t")
#
# cap.release()
# cv2.destroyAllWindows()